Goal: apply a DL RNN model on different dataset
- Preprocess with MinMaxScaler
- Preprocess with CustomerTransformer

# Imports, variables, functions

In [5]:
import scipy.io
from pyedflib import highlevel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
import csv
import pickle
from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
from scipy.signal import freqz, iirnotch, filtfilt
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator
import random
from sklearn.model_selection import cross_validate
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from scipy.fftpack import rfft
from sklearn.metrics import accuracy_score
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from plot_keras_history import plot_history

In [6]:
sample_rate = sampling_rate = 256
sec = 10
len_window = sample_rate*sec
overlap = 5

# Load annotation file
annt = scipy.io.loadmat('../raw_data/annotations_2017.mat')

In [7]:
## -- PREPROCESSING FUNCTIONS --

# Highpass filter
def highpass_filter(signals, sampling_rate, hp_frequency = 0.1):
    sos = butter(N = 3, Wn = hp_frequency, btype="highpass",fs=sampling_rate, output="sos")
    filter_hp = sosfiltfilt(sos, signals)
    return filter_hp

# Powerline filter
def notch_filter(signals, sampling_rate, notch_frequency = 50, quality_factor = 30):
    w0 = notch_frequency/(sampling_rate/2)
    b_notch, a_notch = iirnotch(w0, quality_factor)
    filter_notch = filtfilt(b_notch, a_notch, signals, axis = -1)
    return filter_notch

# Create our own scaler
class CustomTranformer(TransformerMixin, BaseEstimator): 
    # BaseEstimator generates the get_params() and set_params() methods that all Pipelines require
    # TransformerMixin creates the fit_transform() method from fit() and transform()
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.means = X.mean()
        return self
    
    def transform(self, X, y=None):
        norm_features = X - self.means
        return norm_features

# Combination of all filters and Scaler
def filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30):
    filter_hp = highpass_filter(signals, sampling_rate)
    filter_notch = notch_filter(signals, sampling_rate, notch_frequency, quality_factor)
    final_signal = scaler.fit_transform(filter_notch)
    return final_signal

In [8]:
## -- LABEL FUNCTIONS --

# Format the EEG 
def eeg_formated(signals, names_ele):
    data_signals = signals.T # transpose the signals from datapoints
    data_signals = pd.DataFrame(data_signals) # create a pandas dataframe
    
    data_signals.columns = names_ele # rename columns
    
    return data_signals

# Format the annotations
def diagnosis(n):
    patient_A=annt["annotat_new"][0][n-1][0]
    patient_B=annt["annotat_new"][0][n-1][1]
    patient_C=annt["annotat_new"][0][n-1][2]
    
    #converting seconds to datapoints

    patient_A=patient_A.tolist()
    patient_B=patient_B.tolist()
    patient_C=patient_C.tolist()
    
    patient_A_dtp=[]
    patient_B_dtp=[]
    patient_C_dtp=[]  
    for elem in patient_A:
        for i in range(sampling_rate):
            patient_A_dtp.append(elem) 
    for elem in patient_B:
        for i in range(sampling_rate):
            patient_B_dtp.append(elem)
        
    for elem in patient_C:
        for i in range(sampling_rate):
            patient_C_dtp.append(elem)
            
    target_=pd.DataFrame({"Diagnosis A":patient_A_dtp,"Diagnosis B":patient_B_dtp,"Diagnosis C":patient_C_dtp})
    
    return target_  

# Add a time column with the seconds
def add_time(df):
    list_time=[]
    for i in range(len(df)):
        list_time.append(i//sampling_rate)
    df["time"]=list_time
    return df

# Create target variables when seizures lasts at least 10
def is_seizure(df):
    
    threshold = sampling_rate*10
    
    df['is_seizure_A'] = df["Diagnosis A"].groupby((df["Diagnosis A"] != df["Diagnosis A"].shift()).cumsum()).transform('size') * df["Diagnosis A"]
    df['is_seizure_A'] = (df['is_seizure_A'] > threshold).astype(int)
    
    df['is_seizure_B'] = df["Diagnosis B"].groupby((df["Diagnosis B"] != df["Diagnosis B"].shift()).cumsum()).transform('size') * df["Diagnosis B"]
    df['is_seizure_B'] = (df['is_seizure_B'] > threshold).astype(int)
    
    df['is_seizure_C'] = df["Diagnosis C"].groupby((df["Diagnosis C"] != df["Diagnosis C"].shift()).cumsum()).transform('size') * df["Diagnosis C"]
    df['is_seizure_C'] = (df['is_seizure_C'] > threshold).astype(int)
    
    return df 

# Create final target
def create_target(df):
    df['is_seizure_target'] = np.where(df['is_seizure_A'] + df['is_seizure_B'] + df['is_seizure_C'] >= 2, 1, 0)
    return df

# Remove useless
def remove_useless_columns(df):
    df.drop(columns=['Diagnosis A', 'Diagnosis B', 'Diagnosis C', 'is_seizure_A', 'is_seizure_B', 'is_seizure_C', 'ECG EKG', 'Resp Effort', 'time'], inplace=True)
    return df

# Final function to label
def label_data(path_raw_data, signals_preprocessed, n):
    
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])] # extract electrode names
    
    eeg_patient = eeg_formated(signals_preprocessed, names_ele) # format the ECG
    eeg_patient.rename(columns={'ECG EKG-REF':'ECG EKG', 'Resp Effort-REF':'Resp Effort'}, inplace=True)
    diagnosis_patient = diagnosis(n) # format the diagnosis
    
    data_patient = pd.merge(left=eeg_patient, right=diagnosis_patient, how='left', left_index=True, right_index=True) # merge ecg and diagnosis
    
    add_time(data_patient)
    is_seizure(data_patient)
    create_target(data_patient)
    remove_useless_columns(data_patient)
    
    return data_patient

In [9]:
def create_data_input(df):
    data = np.array((df.iloc[i:i+len_window+1]) for i in range(0,len(df)-len_window, overlap*sample_rate))
    r=data.shape[0]
    c=data.shape[1]
    data = pd.DataFrame(data.reshape(r,c))
    
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1]
    return X,y

def oversampling(X, y): 
    sm = SMOTE(sampling_strategy='minority', random_state=7)
    X, y = sm.fit_resample(X, y)
    return X, y

In [10]:
def model_pipeline(path_raw_data, scaler, patient_number, Fournier=False):
    
    # Load raw data
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    # Preprocess data 
    signals_preprocessed = filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)
    
    if Fournier == True:
        signals_preprocessed = pd.DataFrame(np.array([abs(rfft(signals_preprocessed[i])) for i in range(len(signals_preprocessed))]))
        
    # Label data
    df = label_data(path_raw_data, signals_preprocessed, patient_number)
    
    return df

In [11]:
df = model_pipeline("../raw_data/eeg5.edf",  MinMaxScaler(), 5)

In [12]:
df

,EEG Fp1-Ref,EEG Fp2-Ref,EEG F7-Ref,EEG F3-Ref,EEG Fz-Ref,EEG F4-Ref,EEG F8-Ref,EEG T3-Ref,EEG C3-Ref,EEG Cz-Ref,EEG C4-Ref,EEG T4-Ref,EEG T5-Ref,EEG P3-Ref,EEG Pz-Ref,EEG P4-Ref,EEG T6-Ref,EEG O1-Ref,EEG O2-Ref,is_seizure_target
0,0.353494,0.793573,1.000000,0.701917,0.393511,0.183450,0.846558,0.579165,0.018523,0.627815,0.863314,0.909277,0.289584,0.190606,0.209212,0.769056,0.731333,0.000000,0.179405,1
1,0.336246,0.862974,1.000000,0.638187,0.376305,0.179248,0.838919,0.609300,0.000000,0.605302,0.854773,0.920654,0.291398,0.199013,0.214853,0.768280,0.729059,0.017021,0.195751,1
2,0.268712,0.919813,1.000000,0.511203,0.356371,0.224427,0.850186,0.618169,0.000000,0.601201,0.863473,0.937578,0.305050,0.191920,0.219181,0.799121,0.757659,0.045152,0.243526,1
3,0.347118,0.805896,1.000000,0.618322,0.364362,0.234248,0.866393,0.616341,0.000000,0.607192,0.876478,0.956812,0.308363,0.211140,0.236517,0.800898,0.770108,0.051896,0.248591,1
4,0.425620,0.772917,1.000000,0.615228,0.351910,0.214835,0.839804,0.504690,0.000000,0.602062,0.856330,0.955345,0.270725,0.206211,0.235480,0.804234,0.765600,0.037186,0.232421,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983291,0.358651,0.000000,0.446580,0.350410,0.431132,0.438771,0.431005,0.601553,0.432301,0.466851,0.486067,1.000000,0.440335,0.429410,0.481188,0.497280,0.360721,0.479454,0.489994,0
983292,0.027400,0.011211,0.029130,0.000000,0.017762,0.040012,0.040446,0.011096,0.022597,0.031731,0.056863,0.153933,0.036740,0.039975,0.053607,0.071216,0.108301,0.072665,0.062631,0
983293,0.040683,0.080363,0.029821,0.027907,0.026126,0.036374,0.037631,0.000000,0.028419,0.028432,0.038594,0.019143,0.034508,0.037605,0.037596,0.044365,0.081376,0.047479,0.040991,0
983294,0.966625,1.000000,0.959231,0.976907,0.964426,0.956003,0.956450,0.953622,0.962633,0.956752,0.946422,0.872591,0.957035,0.956739,0.947944,0.940503,0.937985,0.941369,0.944088,0


In [13]:
def redefine_window_target(window_df):
    if len(np.unique(window_df.iloc[:,-1])) == 1:
        target = window_df.iloc[0,-1]
    elif np.unique(window_df.iloc[:,-1],return_counts=True)[1][1] >= 2*256:
        target = 1
    else:
        target = 0
    t_df = window_df.drop(columns = "is_seizure_target")
    window = pd.DataFrame(np.array(t_df))
    window["Target"] = target
    return window

In [14]:
data = np.array([redefine_window_target(df.iloc[i:i+len_window+1]) for i in range(0,len(df)-len_window, overlap*sample_rate)])
data.shape

(767, 2561, 20)

In [15]:
data

array([[[3.53494062e-01, 7.93573255e-01, 1.00000000e+00, ...,
         0.00000000e+00, 1.79405397e-01, 1.00000000e+00],
        [3.36246299e-01, 8.62974373e-01, 1.00000000e+00, ...,
         1.70209856e-02, 1.95750725e-01, 1.00000000e+00],
        [2.68711671e-01, 9.19812579e-01, 1.00000000e+00, ...,
         4.51515001e-02, 2.43526078e-01, 1.00000000e+00],
        ...,
        [8.75534928e-02, 1.66695863e-01, 9.94047856e-02, ...,
         7.10290788e-01, 1.63347397e-01, 1.00000000e+00],
        [6.18149653e-02, 1.83765118e-01, 9.64423588e-02, ...,
         8.66426084e-01, 1.88153130e-01, 1.00000000e+00],
        [5.43778494e-02, 1.70836409e-01, 3.18727842e-02, ...,
         8.79038209e-01, 2.35308625e-01, 1.00000000e+00]],

       [[8.29088616e-01, 8.30583388e-01, 4.69496251e-01, ...,
         2.73895195e-01, 4.87277941e-01, 1.00000000e+00],
        [8.40356909e-01, 8.68702377e-01, 4.62443568e-01, ...,
         2.53164669e-01, 4.67961170e-01, 1.00000000e+00],
        [8.51128389e-01, 

In [25]:
#r=data.shape[0]*data.shape[1]
#c=data.shape[2]
    
#data = pd.DataFrame(data.reshape(r,c))
X = data[:,:,:-1]
y = data[:,:,-1]


In [26]:
y=y.sum(axis=1)>=2*256

In [27]:
y

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
def oversampling(X, y): 
    sm = SMOTE(sampling_strategy='minority', random_state=7)
    X, y = sm.fit_resample(X, y)
    return X, y

In [28]:
train_ind, test_ind = train_test_split(np.arange(767), test_size=0.2)

X_train = X[train_ind,:,:]
X_test = X[test_ind,:,:]
y_train = y[train_ind]
y_test = y[test_ind]

#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
    
    # Balancing

#X_train, y_train = oversampling(X_train, y_train)
#y_train.transpose()

In [23]:
y_train

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True,  True,  True, False, False,  True, False, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True, False,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True, False,  True, False,  True,  True,  True,  True,  True,
        True, False,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,

In [20]:
X_train.shape #n.reshape((767, 2561, 19))

(613, 2561, 19)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.SimpleRNN(units=10, activation='tanh',input_shape=(2561, 19)))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# The compilation
model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy', 
        metrics=[ts.keras.metrics.Recall(),"accuracy"])

# The fit
model.fit(X_train, y_train,
         batch_size=32,
         epochs=50, verbose=0)

2022-12-02 17:08:16.287062: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-02 17:08:16.287311: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 17:08:16.287361: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-02 17:08:16.287402: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-02 17:08:16.287441: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [22]:
results_DL =model.evaluate(X_test, y_test)
loss=results_DL[0]
recall = results_DL[1]
accuracy = results_DL[2]
print(loss,recall,accuracy)

5/5 [==============================] - 1s 106ms/step - loss: 0.4387 - recall: 1.0000 - accuracy: 0.8442
0.4387163519859314 1.0 0.8441558480262756
